In [9]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img


In [10]:
main_dir = r"C:\Users\Verneri\Desktop\koulu\iot\iot_project\machine learning\dataset\archive\human detection dataset"


In [11]:
# Load and preprocess your dataset
def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path, 1)
    image = cv2.resize(image, (224, 224))  # Resize to the input size of MobileNetV2
    return image

In [12]:
data = []
labels = []

for class_name in os.listdir(main_dir):
    class_dir = os.path.join(main_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = read_and_preprocess_image(image_path)
        data.append(image)
        labels.append(int(class_name))  # Assuming class names are 0 and 1

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=2)

In [6]:
len(data[0])

224

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [8]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

736
185
736
185


In [14]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [15]:
# Model architecture
model = Sequential()
base_model = MobileNetV2(input_shape=(224, 224, 3))
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Train the model with data augmentation
batch_size = 32
epochs = 20

history = model.fit(datagen.flow(np.array(X_train), y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(np.array(X_test), y_test))

c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.7192 - loss: 0.5660 - val_accuracy: 0.5676 - val_loss: 0.7657
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8526 - loss: 0.3567 - val_accuracy: 0.5676 - val_loss: 0.7604
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.8060 - loss: 0.3890 - val_accuracy: 0.5676 - val_loss: 0.7628
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.8999 - loss: 0.2496 - val_accuracy: 0.5676 - val_loss: 0.8584
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8714 - loss: 0.3056 - val_accuracy: 0.5676 - val_loss: 0.8612
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8687 - loss: 0.3727 - val_accuracy: 0.5676 - val_loss: 0.8585
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9036 - loss: 0.2792 - val_accuracy: 0.5676 - val_loss: 1.0260
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.9036 - loss: 0.2633 - val_accuracy: 0.5676 - val_loss:

In [8]:
# Save the trained model to TensorFlow Lite format
import tensorflow as tf
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
model.save('human_detection_model')
# Save the TensorFlow Lite model to a file
with open('human_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

NameError: name 'model' is not defined

In [18]:
import tensorflow as tf
model.save('human_detection_model.keras')


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=human_detection_model.

In [4]:
import tensorflow as tf

# Load the TFLite model
tflite_model_path = r'machine_learning\human_detection_model.tflite'
tflite_model = tf.lite.Interpreter(model_path=tflite_model_path)

# Convert TFLite to TensorFlow SavedModel
saved_model_path = tflite_model_path
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.experimental_new_converter = True  # Use the new TFLite converter

# Convert and save the model
tf_saved_model = converter.convert()
with tf.io.gfile.GFile('path/to/your/converted_model/saved_model.pb', 'wb') as f:
    f.write(tf_saved_model)

ValueError: Could not open 'machine_learning\human_detection_model.tflite'.

In [6]:
import tensorflow as tf

# Load the TFLite model
tflite_model_path = r'C:\Users\Verneri\Desktop\koulu\iot\iot_project\machine_learning\human_detection_model.tflite'
tflite_model = tf.lite.Interpreter(model_path=tflite_model_path)

OSError: SavedModel file does not exist at: C:\Users\Verneri\Desktop\koulu\iot\iot_project\machine_learning\human_detection_model.tflite\{saved_model.pbtxt|saved_model.pb}